## LDA（Latent Dirichlet Allocation）

トピックモデルと呼ばれる手法の一つ。文書集合に潜在するトピックを推定し、それぞれの文書が持つトピックの割合の結果を用いてクラスタリングする。  
文書が作られる過程として、  

「文書のトピックが決まり、それによって単語が決まる」  

というものを仮定する。  
上記の過程の全体を確立分布としてモデル化し、その確率分布のパラメータをデータから推定していく。

### ライブラリのインストール

In [3]:
# gemsimライブラリのインストール、統計的な計算を用いる自然言語処理に便利な機能を提供している
!pip3 install gensim

import itertools
import json
import logging
import math

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

# https://www.sejuku.net/blog/66459
# システムに関する処理をまとめたライブラリのsysを読み込む
import sys
# 下記でライブラリを読み込めるパス一覧を表示できる。ここにパスを書き込むと異なる階層からライブラリを読み込む事が可能となる。
print(sys.path)
# sys.path.append("相対パス")でsys.pathに追加、ここではディレクトリまでを指定する
sys.path.append("src")

from annoutil import find_xs_in_y
import sqlitedatastore as datastore

    100% |████████████████████████████████| 24.2MB 71kB/s eta 0:00:011  3% |█                               | 839kB 6.1MB/s eta 0:00:04    29% |█████████▋                      | 7.3MB 7.7MB/s eta 0:00:03    39% |████████████▌                   | 9.5MB 6.2MB/s eta 0:00:03    45% |██████████████▊                 | 11.1MB 6.8MB/s eta 0:00:02    80% |█████████████████████████▋      | 19.4MB 8.7MB/s eta 0:00:01
    100% |████████████████████████████████| 20.0MB 88kB/s  eta 0:00:011 7% |██▎                             | 1.5MB 7.5MB/s eta 0:00:03    20% |██████▌                         | 4.0MB 3.3MB/s eta 0:00:05    53% |█████████████████               | 10.7MB 10.2MB/s eta 0:00:01    59% |███████████████████             | 11.8MB 8.4MB/s eta 0:00:01    88% |████████████████████████████▎   | 17.7MB 24.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://

In [ ]:
# LDAの計算過程を表示する為に、loggingモジュールでログの表示設定をする
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

datastore.connect()
sentences = []
# コーパスとして文ごとに単語の原型のリストをsents変数に格納する
for doc_id in datastore.get_all_ids(limit=-1):
    all_tokens = datastore.get_annotation(doc_id, 'token')
    for sent in datastore.get_annotation(doc_id, 'sentence'):
        tokens = find_xs_in_y(all_tokens, sent)
        
        sentences.append([token['lemma'] for token in tokens if token.get('NE') ==  'O'])